## Week 2 Day 2

Our first Agentic Framework project using the Google Gemini API!

We're going to build a simple system for generating cold sales outreach emails:
1. Generating multiple email styles.
2. Using tools to call functions (like sending an email).
3. Orchestrating a multi-step workflow.

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

Please set up an account - it's free!

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [ ]:
from dotenv import load_dotenv
from google import genai
from google.genai import types
from typing import Dict, List
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [ ]:
load_dotenv(override=True)

In [ ]:
# Let's just check emails are working for you

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("test@example.com")  # Change to your verified sender
    to_email = To("test@example.com")  # Change to your recipient
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

# send_test_email() # Commented out to avoid running automatically

### Did you receive the test email

If you get a 202, then you're good to go!

## Step 1: Generating Email Variations

In [ ]:
client = genai.Client()

instructions1 = "You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write concise, to the point cold emails."

In [ ]:
async def generate_email(instructions: str, prompt: str) -> str:
    """Generates an email using the Gemini API with specific instructions."""
    response = client.models.generate_content(
        model='gemini-2.5-flash',
        contents=[prompt],
        config=types.GenerateContentConfig(
            system_instruction=instructions
        )
    )
    return response.text

async def generate_in_parallel():
    message = "Write a cold sales email"
    tasks = [
        generate_email(instructions1, message),
        generate_email(instructions2, message),
        generate_email(instructions3, message),
    ]
    results = await asyncio.gather(*tasks)
    
    for i, output in enumerate(results):
        print(f"--- Email Style {i+1} ---\n{output}\n")

await generate_in_parallel()

In [ ]:
async def select_best_email():
    message = "Write a cold sales email"
    tasks = [
        generate_email(instructions1, message),
        generate_email(instructions2, message),
        generate_email(instructions3, message),
    ]
    outputs = await asyncio.gather(*tasks)

    emails_for_selection = "Here are three cold sales emails:\n\n" + "\n\n---\n\n".join(outputs)
    
    picker_instructions = "You pick the best cold sales email from the given options. Imagine you are a customer and pick the one you are most likely to respond to. Do not give an explanation; reply with the selected email only."

    best_email_response = await generate_email(picker_instructions, emails_for_selection)
    
    print(f"--- Best Sales Email ---\n{best_email_response}")

await select_best_email()

## Part 2: Use of Tools for Function Calling

In [ ]:
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    try:
        sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
        from_email = Email("test@example.com")  # Change to your verified sender
        to_email = To("test@example.com")  # Change to your recipient
        content = Content("text/plain", body)
        mail = Mail(from_email, to_email, "Sales email", content).get()
        sg.client.mail.send.post(request_body=mail)
        return {"status": "success"}
    except Exception as e:
        return {"status": "error", "message": str(e)}


## Part 3: Orchestrating a Sales Agent Workflow

Now we'll create a multi-step process that uses other model calls as 'tools'.

In [ ]:
def write_professional_email(prompt: str) -> str:
    """Generates a professional sales email."""
    return client.models.generate_content(
        model='gemini-2.5-flash',
        contents=[prompt],
        config=types.GenerateContentConfig(system_instruction=instructions1)
    ).text

def write_engaging_email(prompt: str) -> str:
    """Generates an engaging and humorous sales email."""
    return client.models.generate_content(
        model='gemini-2.5-flash',
        contents=[prompt],
        config=types.GenerateContentConfig(system_instruction=instructions2)
    ).text

def write_concise_email(prompt: str) -> str:
    """Generates a concise, to-the-point sales email."""
    return client.models.generate_content(
        model='gemini-2.5-flash',
        contents=[prompt],
        config=types.GenerateContentConfig(system_instruction=instructions3)
    ).text

In [ ]:
async def sales_manager_workflow():
    """Orchestrates the entire sales email generation and sending process."""
    
    manager_instructions = """
    You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email.
    
    Follow these steps carefully:
    1. Generate Drafts: Use all three email writing tools to generate three different email drafts.
    2. Evaluate and Select: Review the drafts and choose the single best email.
    3. Send Email: Use the send_email tool to send the best email.
    
    You must use the tools. Do not write the emails yourself. You must only send ONE email.
    """
    
    message = "Send a cold sales email addressed to 'Dear CEO'"
    
    tools = [write_professional_email, write_engaging_email, write_concise_email, send_email]
    
    response = client.models.generate_content(
        model='gemini-2.5-flash',
        contents=[message],
        config=types.GenerateContentConfig(system_instruction=manager_instructions, tools=tools)
    )
    
    print("Initial response from manager model:", response.text)

    # This is a simplified orchestration. A real-world scenario would involve a loop
    # to handle multiple tool calls until the task is complete.
    if response.function_calls:
        print("\n--- Function Calls Requested ---")
        for fc in response.function_calls:
            print(f"Function: {fc.name}, Args: {dict(fc.args)}")
    
    # In a real application, you would now execute these functions and send the results back to the model.

await sales_manager_workflow()

## Note on Gemini Function Calling

The above example demonstrates how to make functions available to the Gemini model. The model doesn't execute the functions itself but instead returns a `function_call` object telling us which function to run and with what arguments. 

A complete implementation would involve a loop:
1. Call `generate_content` with tools.
2. If the response contains function calls, execute them using your Python code.
3. Send the results from the function execution back to the model in a new `generate_content` call.
4. Repeat until the model provides a final text response instead of a function call.